<h1>Regresión Logística</h1>

En este Notebook vamos a hacer una regresión logístia por medio de una sola neurona, o perceptró. El proceso va a ser llevado desde un inicio con el mayor nivel de explicación que me sea posible dar.

El ejercicio está basado en el Notebook de Andrew Ng llamado Logistic Regression with a Neural Network, aunque el ejercicio está excelentemente explicado quiero replicarlo, esto lo voy a hacer en mis propias palabras es por eso que lo estoy haciendo en español.

LLamamos nuestras librerias a ocupar.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import pandas as pd 
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("/content/sample_data/divorce.csv", sep = '\t', header=None)

In [3]:
# dataframe de divorciados
df_div = df[df[54] == 1]
# dataframe de los casados
df_cas = df[df[54] == 0]

In [4]:
target_div = df_div[54]
target_cas = df_cas[54]

In [5]:
df_div = df_div.iloc[:, :-1]
df_cas = df_cas.iloc[:, :-1]

In [6]:
print(df_div.shape)
print(df_cas.shape)


(84, 54)
(86, 54)


In [7]:
X_train_div, X_test_div, y_train_div, y_test_div = train_test_split(df_div, target_div, train_size=0.8)
X_train_cas, X_test_cas, y_train_cas, y_test_cas = train_test_split(df_cas, target_cas, train_size=0.8)

In [8]:
print(X_train_div.shape)
print(X_test_div.shape)
print(y_train_div.shape)
print(y_test_div.shape)
print()
print(X_train_cas.shape)
print(X_test_cas.shape)
print(y_train_cas.shape)
print(y_test_cas.shape)


(67, 54)
(17, 54)
(67,)
(17,)

(68, 54)
(18, 54)
(68,)
(18,)


In [9]:
X_train = pd.concat((X_train_div, X_train_cas), axis=0)
X_test =  pd.concat((X_test_div , X_test_cas), axis=0)
y_train = pd.concat((y_train_div, y_train_cas), axis=0)
y_test =  pd.concat((y_test_div,  y_test_cas), axis=0)

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(135, 54)
(35, 54)
(135,)
(35,)


In [11]:
# Sacamos el numpy de cada dataframe y lo guardamos con su 
# traspusta para acomodarlo en el convenio
X_train = X_train.values
X_train = X_train.T

X_test = X_test.values
X_test = X_test.T

In [12]:
# Normalizamos el conjunto X de entrenamiento y de test
# como el valor máximo que podemos tener es 4 solo vamos a dividir entre en máximo
X_train = X_train / 4
X_test = X_test / 4


In [15]:
# Como podemos ver el valor de m en nuestro ejemplo es 135
# y el tamaño del conjunto de prueba es de 35
print(X_train.shape)
print(X_test.shape)

(54, 135)
(54, 35)


In [16]:
y_train = y_train.values
y_test = y_test.values
print(y_train.shape)
print(y_test.shape)


(135,)
(35,)


In [19]:
y_train = y_train.reshape((1, y_train.shape[0]))
y_test = y_test.reshape((1, y_test.shape[0]))

In [20]:
print(y_train.shape)
print(y_test.shape)

(1, 135)
(1, 35)


Creamos las funciones necesarias para ejecutar la
regresión logística.


In [21]:
# Función de activación
def sigmoid(z):
    """
    Computa el sigmoide de z

    Argumentos:
    z -- Un escalar o un array de numpy de cualquier tamaño.

    Returna:
    s -- sigmoid(z)
    """
    s = 1 / (1 + np.exp(-z))
    return s

In [22]:
# Función de inicialización de pesos en ceros
def initialize_with_zeros(dim):
    """
    Esta función crea un vector de ceros con dimensiones (dim, 1) para w e inicializa b a cero
    
    Argument:
    dim -- tamaño de el vector w que queremos (o el número de parámetros en este caso)
    size of the w vector we want (or number of parameters in this case)
    
    Returna:
    w -- el vector inicializado de dimensiones (dim, 1)
    b -- el escalar inicializado de dimensiones (dim, 1)
    """
    w = np.zeros(shape=(dim, 1))
    b = 0
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    return w, b

In [23]:
# Función de propagación
def propagate(w, b, X, Y):
    """
    Implementa la función costo y su gradiente para la propagación explicada anteriormente

    Argumentos:
    w -- weights o pesos, un array numpy de tamaño (num_atributos[54 en este caso], 1) 
    b -- bias o sesgo, un escalar
    X -- datos con el tamaño (num_atributos[54 en este caso], numbero de ejemplos)
    Y -- vector "etiqueta" verdad(contiene 0 si esta casado, 1 si está divorsiado) de tamaño (1, numero de ejemplos)

    Returna:
    cost -- la probabilidad del costo logarítmico negativo para la regresión logística
    dw -- gradiente de la perdida con respecto a w, así tiene la misma dimensión de w 
    db -- gradiente de la perdida con respecto a b, así tiene la misma dimensión de b    
    """
    
    m = X.shape[1]
    
    # PROPAGACIÓN HACIA DELANTE (DESDE X A COST)
    A = sigmoid(np.dot(w.T, X) + b)                         # computa la activación
    cost = np.sum( Y*np.log(A) + (1-Y)*np.log(1-A) ) / -m   # computa el cost
    
    # PROPAGACIÓN HACIA ATRÁS (PARA ENCONTRAR EL GRADIENTE)
    dw = np.dot(X, (A-Y).T) / m
    db = np.sum(A-Y) / m

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [24]:
# FUNCIÓN: optimizar

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    Esta función optimiza w y b por correr un algoritmo de gradiente descendiente
    
    Argumentos:
    w -- weights o pesos, un array numpy de tamaño (número de atributos, 1)
    b -- bias o sesgo, un escalar
    X -- datos con el tamaño (num_atributos[54 en este caso], numbero de ejemplos)
    Y -- vector "etiqueta" verdad(contiene 0 si esta casado, 1 si está divorsiado) de tamaño (1, numero de ejemplos)
    num_iterations -- número de iteraciones del ciclo de optimización
    learning_rate -- tasa de aprendizaje de la regla de actualización del gradiente descendiente
    print_cost -- Si es True imprime la pérdida cada 100 pasos 
    
    Returna:
    params -- diccionario que contiene los pesos w y el sesgo b
    grads -- diccionario que contiene los gradiente de los pesos y el sesgo respecto a la función cost
    costs -- lista de todos los costos computados durante la optimización, esto será usado para graficar la curva de aprendizaje     
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        grads, cost = propagate(w, b, X, Y)

        #  Resupera las derivadas desde grads
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate*dw
        b = b - learning_rate*db
        
        # Guanta los costos
        if i % 100 == 0:
            costs.append(cost)
        
        # Imprime los costos cada 100 iteraciones de entrenamiento
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [25]:
# FUNCIÓN: predecir

def predict(w, b, X):
    '''
    Predice ya sea si la etiqueta es 0 o 1 usando los parámetros de regresión logística aprendidos (w, b) 
    
    Argumentos:
    w -- weights o pesos, un array numpy de tamaño (número atributos, 1)
    b -- bias o sesgo, un escalar
    X -- datos de tamño (número atributos, número de ejemplos)
    
    Returna:
    Y_prediction -- un array numpy (vector) conteniendo todas las predicciones (0/1) para los ejemplos en X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # Computa el vector A prediciendo las probabilidades de que esté divorciado en un evento
    A = sigmoid(np.dot(w.T, X) + b)
    
    for i in range(A.shape[1]):
        
        # Convierte las probabilidades A[0, i] a las predicciones actuales p[0, i]
        if A[0][i] <= 0.5:
            Y_prediction[0][i] = 0
        else:
            Y_prediction[0][i] = 1
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [26]:
# FUNCIÓN: modelo

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    """
    Construye el modelo de regresión logística llamando las funciones que implementamos anteriormente
    
    Argumentos:
    X_train -- el conjunto de entrenamiento representado por un array numpy de tamaño (número atributos, m_train)
    Y_train -- las etiquetas de entrenamiento representadas por un array numpy (vector) de tamaño (1, m_train)
    X_test -- conjunto de prueba representadas por un array numpy de tamaño (número atributos, m_test)
    Y_test -- etiquetas de la prueba representadas por un array numpy (vector)de tamaño (1, m_test)
    num_iterations -- hiperparámetro representando los números de iteraciones para optimizar los parámetros
    learning_rate -- hiperparámetro representando la tasa de aprendizaje usado en la regla de actualizacióón de optimize()
    print_cost -- puesto en true imprime los costos cada 100 iteraciones
    
    Returna:
    d -- dictionario conteniendo información sobre el modelo.
    """

    # incializa los parámetros con ceros    
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradiente descendiente 
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # Recupera los parámetros w y b desde el dictionario "parametros"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predice los conjuntos prueba/entrenamiento 
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    # imprime los errores de entrenamiento/prueba
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [27]:
d = model(X_train, y_train, X_test, y_test, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.487099
Cost after iteration 200: 0.428639
Cost after iteration 300: 0.384439
Cost after iteration 400: 0.348096
Cost after iteration 500: 0.317759
Cost after iteration 600: 0.292211
Cost after iteration 700: 0.270514
Cost after iteration 800: 0.251937
Cost after iteration 900: 0.235904
Cost after iteration 1000: 0.221960
Cost after iteration 1100: 0.209748
Cost after iteration 1200: 0.198981
Cost after iteration 1300: 0.189430
Cost after iteration 1400: 0.180909
Cost after iteration 1500: 0.173267
Cost after iteration 1600: 0.166379
Cost after iteration 1700: 0.160143
Cost after iteration 1800: 0.154473
Cost after iteration 1900: 0.149298
train accuracy: 97.77777777777777 %
test accuracy: 97.14285714285714 %


In [28]:
for enu, i in enumerate(d['Y_prediction_test'].T):
    print(enu, i)
print(type(d['Y_prediction_test']))
    

0 [1.]
1 [1.]
2 [1.]
3 [1.]
4 [1.]
5 [0.]
6 [1.]
7 [1.]
8 [1.]
9 [1.]
10 [1.]
11 [1.]
12 [1.]
13 [1.]
14 [1.]
15 [1.]
16 [1.]
17 [0.]
18 [0.]
19 [0.]
20 [0.]
21 [0.]
22 [0.]
23 [0.]
24 [0.]
25 [0.]
26 [0.]
27 [0.]
28 [0.]
29 [0.]
30 [0.]
31 [0.]
32 [0.]
33 [0.]
34 [0.]
<class 'numpy.ndarray'>


In [29]:
for enu, i in enumerate(y_test.T):
    print(enu, i)

0 [1]
1 [1]
2 [1]
3 [1]
4 [1]
5 [1]
6 [1]
7 [1]
8 [1]
9 [1]
10 [1]
11 [1]
12 [1]
13 [1]
14 [1]
15 [1]
16 [1]
17 [0]
18 [0]
19 [0]
20 [0]
21 [0]
22 [0]
23 [0]
24 [0]
25 [0]
26 [0]
27 [0]
28 [0]
29 [0]
30 [0]
31 [0]
32 [0]
33 [0]
34 [0]
